In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
df = pd.read_excel(r'C:\Users\heygu\과제_파일.xlsx')
print("Train subset column : {}".format(df.columns))

## 시간 데이터 변환
- 전처리하기 쉽도록 데이터 변환
- 월, 년, 주로 분리하여 저장

In [ ]:
df['start_date'] = pd.to_datetime(df['start_date'], format='%Y-%m-%d %H:%M:%S') # object -> datetime
df['start_Year'] = df['start_date'].dt.year # 연
df['start_Month'] =df['start_date'].dt.month # 월
df['start_Day'] =df['start_date'].dt.day # 일
df['strart_WeekOfYear'] = df['start_date'].dt.strftime("%W")
df['strart_WeekOfYear'] = pd.to_numeric(df['strart_WeekOfYear'])

df['end_date'] = pd.to_datetime(df['end_date'], format='%Y-%m-%d %H:%M:%S') # object -> datetime
df['end_Year'] = df['end_date'].dt.year # 연
df['end_Month'] =df['end_date'].dt.month # 월
df['end_Day'] =df['end_date'].dt.day # 일
df['end_WeekOfYear'] = df['end_date'].dt.strftime("%W")
df['end_WeekOfYear'] = pd.to_numeric(df['end_WeekOfYear'])

df = df.sort_values(by='start_date')

## 결측치 처리 하기
- non-null에 값은 동일하다.하지만 그렇다고 결측치가 없다고 할 수 없다.
- 형에도 이상이 없다.
- 년, 월, 일, 주에 데이터 형식이나 이상값이 있는지 확인하기 (데이터의 범위, 형식 등)

In [ ]:
df.info()
df.isnull().sum()

In [ ]:
import missingno as msno
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 결측값 영역 표시
msno.matrix(df)
plt.show()

# 결측값 막대 그래프
msno.bar(df)
plt.show()

In [ ]:
# 결측값이 아닌 빈 문자열이 있는지 확인
# 애초에 연속형이라 있을 수가 없음.
def is_emptystring(x):
    return x.eq('').any()
 
df.apply(lambda x:is_emptystring(x))

### 특성 독립 변수 결측치
- revenue 주간 매출
- total_active_days 모든 유저의 활성 일수 총합
- total_time 모든 유저의 플레이 시간 총합 
- average_session_duration 실행시 평균 앱 실행 시간
- average_session_per_user 주간 평균 앱 실행 횟수
- average_time_per_user 주간 평균 1인당 플레이 시간

In [ ]:
#년,월,일,주에 년도가 두자리 라던거 말이 안되는 값이 있는지 확인하기
df.start_Year.value_counts()
df.end_Year.value_counts()
df.start_Month.value_counts()
df.start_Day.value_counts()
df.end_Week.value_counts()
df.start_Week.value_counts()
df.end_Month.value_counts()
df.end_Day.value_counts()

In [ ]:
#1000이상인 이상한 값 있는지 확인
df.product_id.value_counts()

In [ ]:
#df.average_active_days.value_counts()
#df.average_active_users.value_counts()
#df.average_session_duration.value_counts()
#df.average_time_per_user.value_counts()
#df.average_session_per_user.value_counts()

#df.download.value_counts()
#df.install_base.value_counts()
#df.open_rate.value_counts()
#df.revenue.value_counts()
#df.total_active_days.value_counts()
#df.total_time.value_counts()
#df.genre.value_counts()

In [1]:
df_revence_miss = df[(df["revenue"] <= 0)]
pd.set_option('display.max_columns', None)
df_revence_miss
len(df_revence_miss)

NameError: name 'df' is not defined

### 변수간 수학적 계산을 통한 결측치 처리

### 결측치 대체 기준
- 삭제하기에는 값이 일주일치가 사라지는 효과가 생기기에 위험도가 따른다.

#### revenue 결측치
- 매출이란? 유료서비스나 아이템, 유료 앱등을 구매했을 때 수익이 발생하면서 생기는 것
- 정확히 구매자 데이터가 없기 때문에 유저가 플레이만 했는지 구매를 했는지 알수가 없다.
- 실질적인 구매가 없었다면 수익은 발생하지 않아서 0 인것이 이해가 된다.
- 688개로 아주 많은 양을 차지하고 있다고 느껴지지 않기에 넘어가겠다.
- 추후 문제가 될 경우 다시 확인해보기


#### 변수간에 수학적 계산을 통해서 처리 하려했으나 비식별화가 되어있어 그러기는 어렵다

- 중앙값으로 대체하기

- 모두 0인 경우 결측치 아님 그냥 유저가 없었을 뿐이다.<br>

- 실행시 평균 앱 실행 시간 : average_time_per_user 주간 평균 1인당 플레이 시간 , total_time 모든 유저의 플레이 시간 총합 

- 모든 유저의 활성 일수 총합 : average_active_days 주간 평균 활성 일수, average_session_duration 실행시 평균 앱 실행 시간, average_session_per_user 주간 평균 앱 실행 횟수, average_time_per_user 주간 평균 1인당 플레이 시간

- total_time : average_time_per_user 주간 평균 1인당 플레이 시간, average_session_duration 실행시 평균 앱 실행 시간 (한 번 실행 했을 때 평균적으로 플레이하는 시간)

- 주간 평균 앱 실행 횟수 : average_time_per_user 주간 평균 1인당 플레이 시간, total_time 모든 유저의 플레이 시간 총합, average_session_duration 실행시 평균 앱 실행 시간 (한 번 실행 했을 때 평균적으로 플레이하는 시간),

- 주간 평균 1인당 플레이 시간 : average_time_per_user 주간 평균 1인당 플레이 시간, total_time 모든 유저의 플레이 시간 총합

In [ ]:
for num in range(0,54609):
    if df.loc[num ,"average_time_per_user"] == 0 :
        if ((df.loc[num ,"total_time"] > 0) | (df.loc[num ,"average_session_duration"] > 0)):
            df_new = df[df.loc[num, "product_id"] == df['product_id']]
            df.loc[num ,"average_time_per_user"] = df_new["average_time_per_user"].median()
            
    if df.loc[num ,"average_session_per_user"] == 0:
        if((df.loc[num ,"total_time"] > 0) | (df.loc[num ,"average_session_duration"] > 0)):
            df_new = df[df.loc[num, "product_id"] == df['product_id']]
            df.loc[num ,"average_session_per_user"] = df_new["average_session_per_user"].median()
            
    if df.loc[num ,"average_session_duration"] == 0:
        if((df.loc[num ,"total_time"] > 0) | (df.loc[num ,"average_session_per_user"] > 0)):
            df_new = df[df.loc[num, "product_id"] == df['product_id']]
            df.loc[num ,"average_session_duration"] = df_new["average_session_duration"].median()
            
    if df.loc[num ,"total_time"] == 0:
        if ((df.loc[num ,"average_session_duration"] > 0) | (df.loc[num ,"average_session_per_user"] > 0 )| (df.loc[num ,"average_time_per_user"] > 0)):
            df_new = df[df.loc[num, "product_id"] == df['product_id']]
            df.loc[num ,"total_time"] = df_new["total_time"].median()
            
    if df.loc[num ,"total_active_days"] == 0:
        if (df.loc[num ,"average_active_days"] > 0):
            df_new = df[df.loc[num, "product_id"] == df['product_id']]
            df.loc[num ,"average_active_days"] = df_new["average_active_days"].median()

### 비식별화로 인해 제외된 산술계산 방식

In [ ]:
for num in range(0,54609):
    if (df.loc[num ,"average_time_per_user"] == 0):
        if((df.loc[num ,"total_time"] > 0) & (df.loc[num ,"average_session_duration"] > 0)):
            print(df.loc[num ,"total_time"], df.loc[num ,"average_active_users"])
            value1 = (df.loc[num ,"total_time"]/df.loc[num ,"average_session_duration"])/7
        #value2 = df.loc[num ,"open_rate"]*df.loc[num, "install_base"]*(df.loc[num ,"average_time_per_user"]*7)
        #value3 = (df.loc[num ,"average_active_users"]*7)*(df.loc[num ,"average_time_per_user"]*7)
        if value1 > 0:
            df.loc[num, "average_session_per_user"] = value1
            value1 = 0
      #  elif value2 > 0:
      #      df.loc[num, "total_time"] = value2
      #  elif value3 > 0:
      #      df.loc[num, "total_time"] = value3

In [ ]:
for num in range(0,54609):
    if (df.loc[num ,"average_session_per_user"] == 0):
        if((df.loc[num ,"total_time"] > 0) & (df.loc[num ,"average_session_duration"] > 0)):
            print(df.loc[num ,"total_time"], df.loc[num ,"average_session_duration"])
            value1 = df.loc[num ,"total_time"]/(df.loc[num ,"average_session_duration"])
        #value2 = df.loc[num ,"open_rate"]*df.loc[num, "install_base"]*(df.loc[num ,"average_time_per_user"]*7)
        #value3 = (df.loc[num ,"average_active_users"]*7)*(df.loc[num ,"average_time_per_user"]*7)
        if value1 > 0:
            df.loc[num, "average_session_per_user"] = value1
            value1 = 0
      #  elif value2 > 0:
      #      df.loc[num, "total_time"] = value2
      #  elif value3 > 0:
      #      df.loc[num, "total_time"] = value3

In [ ]:
for num in range(0,54609):
    if (df.loc[num ,"average_session_duration"] == 0):
        if((df.loc[num ,"total_time"] > 0) & (df.loc[num ,"average_session_per_user"] > 0)):
            print(df.loc[num ,"total_time"], df.loc[num ,"average_session_per_user"])
            value1 = df.loc[num ,"total_time"]/(df.loc[num ,"average_session_per_user"]*7)
        #value2 = df.loc[num ,"open_rate"]*df.loc[num, "install_base"]*(df.loc[num ,"average_time_per_user"]*7)
        #value3 = (df.loc[num ,"average_active_users"]*7)*(df.loc[num ,"average_time_per_user"]*7)
        if value1 > 0:
            df.loc[num, "average_session_duration"] = value1
            value1 = 0
      #  elif value2 > 0:
      #      df.loc[num, "total_time"] = value2
      #  elif value3 > 0:
      #      df.loc[num, "total_time"] = value3

In [ ]:
for num in range(0,54609):
    if (df.loc[num ,"total_time"] == 0):
        value1 = df.loc[num ,"average_session_duration"]*(df.loc[num ,"average_session_per_user"]*7)
        value2 = df.loc[num ,"open_rate"]*df.loc[num, "install_base"]*(df.loc[num ,"average_time_per_user"]*7)
        value3 = (df.loc[num ,"average_active_users"]*7)*(df.loc[num ,"average_time_per_user"]*7)
        if value1 > 0:
            df.loc[num, "total_time"] = value1
            value1 = 0
        elif value2 > 0:
            df.loc[num, "total_time"] = value2
            value2 = 0
        elif value3 > 0:
            df.loc[num, "total_time"] = value3
            value3 = 0 

In [ ]:
for num in range(0,54609):
    if (df.loc[num ,"total_active_days"] == 0):
        value1 = df.loc[num ,"average_active_days"]*7
        value2 = df.loc[num ,"open_rate"]*df.loc[num, "install_base"]
        #print(df.loc[[num]]["average_active_days"], value1, value2)
        if value1 > 0:
            df.loc[num, "total_active_days"] = value1
            value1 = 0
        elif value2 > 0:
            df.loc[num, "total_active_days"] = value2
            value2 = 0

In [ ]:
df_total_active_days_miss = df[(df["total_active_days"] <= 0)]
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df_total_active_days_miss

### 기간 행 결측치 발견
- 이전 년도가 없거나 
- 몇 달이 없거나 한 경우가 너무 많음
- 104개여야함 
- 추세적으로 비슷한 애들을 묶은 후에 값을 채워줄 필요가 있음 그렇지 않으면 지점별로 데이터가 부족해서 다음 값을 특정할 수가 없다.
- 게다가 가장 긴게 92개이다.
- df[df["product_id"]==185]처럼 심각하게 1개 혹은 2개 밖에 값이 없는 경우도 있다.

##### 해결 방식
- 그룹화하여 그룹화된 애들의 중앙값으로 만들어 버릴 예정
- 단, 너무 많은 결측치가 존재하는 경우 이는 메우지 않는다.
- 시계열은 없는 시간대를 만든다는게 쉽지 않기에 클러스터링 할때만 사용해서 혹여 잘못 훈련하는 상황을 방지 하겠다.
- 2021,2022이 추세가 다른다. 년도는 이어지듯이 매출이 달라진다.
- 되도록 많은 데이터로 해당 추세를 예측하려고 해야한다. 
- 시간에 따라서 일정한 매출을 가지지는 않아보인다. 상당히 튀는 값을 가진다.
- 그 다음년도까지 확실히 영향을 주는걸 연도별 비교로 확인 했음
- 심각하게 손실된 제품은 값을 제거해주고 차라리 다른 애들을 추론한 중앙값으로 대체하겠다.

In [2]:
fig = plt.figure(figsize=(30,60))

for store in range(0,50):
    storeset = df[df.product_id ==store]
    storeset_2021 = storeset[storeset.start_Year==2021]
    storeset_2022 = storeset[storeset.start_Year==2022]
    
    ax = fig.add_subplot(12,5, store + 1)
    
    plt.title(f"store_{store}")
    ax.plot(storeset_2021.end_WeekOfYear, storeset_2021.revenue, label="2021", alpha=0.3)
    ax.plot(storeset_2022.end_WeekOfYear, storeset_2022.revenue, label="2022", alpha=0.3)
    ax.legend()
    
plt.show()

NameError: name 'df' is not defined

### 각 게임별 연도별 매출 확인

In [ ]:
fig = plt.figure(figsize=(30,60))

for store in range(0,50):
    storeset = df[df.product_id ==store]
    storeset_2021 = storeset[storeset.start_Year==2021]
    storeset_2022 = storeset[storeset.start_Year==2022]
    
    ax = fig.add_subplot(12,5, store + 1)
    
    plt.title(f"store_{store}")
    ax.plot(storeset_2021.end_WeekOfYear, storeset_2021.revenue, label="2021", alpha=0.3)
    ax.plot(storeset_2022.end_WeekOfYear, storeset_2022.revenue, label="2022", alpha=0.3)
    ax.legend()
    
plt.show()

In [ ]:
df2 = pd.DataFrame(columns = df.columns)
for num in range(0,1000):
    co = df[(df["product_id"] == num)]
    size = co.count()
    if size.start_date == 92:
        print(co.count(), num)
        df2 = pd.concat([df2,co], axis=0)

# 게임별 매출 추세 확인하기

- 잘 보면 일정하게 급격하게 치솟다가 하강하는 그래프들이 종종 보이는데 모바일 게임일 확률이 높다. 모바일 게임의 경우 게임 출시 초반에 매출이 가장 높고 그 이후에는 점차 감소하게 됩니다

- 게임별 매출 추세가 상당히 다르기 때문에 평균이나 총합으로 처리하면 안될 것 같다.
- 개별적이나 그룹화가 필요하다.

* 게임 특성별로 묶으면 모바일, RPG 등으로 보는 편이 좋을 것 같다. 경향성이 비슷한 게임끼리 묶어서 계산하는 편이 좋을 것 같다.(군집화 진행)
1) 모바일 게임
2) 온라인 게임
3) 모바일 / 온라인 게임
- 각각에 게임 안에 카테고리 캐주얼, 슈팅 등등

In [ ]:
plt.figure(figsize=(100, 100))
f, axes = plt.subplots(100, 10)
# 격자 크기 설정
f.set_size_inches((300, 300))
# 격자 여백 설정
plt.subplots_adjust(wspace = 0.2, hspace = 0.2)

x = 0;
y = 0;

for num in (range(0,1000)):
    axes[y][x].plot(df[(df["product_id"] == num)].start_date, df[(df["product_id"] == num)].revenue)
    x = x + 1
    if x == 10:
        y = y + 1
        x = 0

#plt.show()
plt.savefig(r'C:\Users\heygu\savefig_default.png')

### 게임별 주간 매출액 합 비교
- 기간 결측치를 고려해서 평균값으로 선정, 합으로 할 경우 결측치로 인해서 심하게 차이남
- 게임별로 매출액의 차이가 크므로 지점별로 모델을 만들어 따로 예측을 해줘야겠다는 생각.

In [ ]:
train_max_store_sum = df.groupby('product_id')[['revenue']].mean()
fig = plt.figure(figsize = (20,12))
sns.barplot(x=train_max_store_sum.index, y=train_max_store_sum['revenue'], data=df)
plt.title('Weekly_sales by store(sum)')
plt.xticks(rotation=90)
plt.show()

### 게임별 매출액과 변수들의 상관관계 확인
- 1000개를 range를 조정해서 여러번 확인해 본 결과
- 그래프 상에 게임별 변수들의 관계가 같지는 않음.
- 물론 일정 부분 download, install_base, open_rate, total_active_days, total_time 이 많은 영향을 대체적으로 주지만 그게 모든 제품에 통용되지 않는 것이 그래프 상에 보임
- 하지만 분명히 일정 게임들은 공통된 상관관계를 가지고 있는 것이 시각적으로 보인다.
- 게임별 혹은 그룹별 모델링의 필요성이 느껴짐

In [ ]:
corr = []
for num in range(0,50):
    co = df[df.product_id==num]
    co = co.reset_index()
    num_corr = co.corr(method='spearman',numeric_only=True)['revenue']
    num_corr = num_corr.drop(['product_id','revenue', 'start_Year', 'start_Month', 'start_Day', 'strart_WeekOfYear'])
    corr.append(num_corr)
corr_df = pd.concat(corr, axis=1).T
corr_df.index = list(range(0,50))
f, ax = plt.subplots(figsize=(30,15))
plt.title("게임별 매출액과 변수들간의 상관관계", fontsize=15)
sns.heatmap(corr_df.T, cmap=sns.diverging_palette(240,10,as_cmap=True), ax=ax, annot = True)
plt.xlabel('게임(id)')
plt.show()


* 또한 실제 도메인 지식을 기반으로 정리한 결과 유의미한 변수 관계라는 판단이 들었다.

이를 통해서 각 변수간의 상관관계가 높은 것들을 추려 낼 수 있었다.
상관성이 높다고 해서 무조건 유의미 하지는 않으므로 주의해야한다.


 ### 정규성 검정
 - 귀무가설 : 해당 데이터가 정규성을 충족한다.
- 특징 : data수가 5000을 초과하는 경우, 정규성 검정에 대한 P 값을 담보하지 못한다. (즉, 통계적 유의성이 떨어진다.)
- 0.05 이상이어야 유의하다.
- 0.05 보다 작다.
- 대부분의 값이 정규성을 띄고 있지 않음
- 정규성이 없기에 비선형적 관계일 가능성이 크기에 method='spearman' 사용한다.

In [ ]:
from scipy import stats
shapiro_test, p_val = stats.shapiro(df_stand["average_active_days"])
print("Test-statistis:{}, p-value:{}".format(shapiro_test, p_val))
shapiro_test, p_val = stats.shapiro(df_stand["average_active_users"])
print("Test-statistis:{}, p-value:{}".format(shapiro_test, p_val))
shapiro_test, p_val = stats.shapiro(df_stand["average_session_duration"])
print("Test-statistis:{}, p-value:{}".format(shapiro_test, p_val))
shapiro_test, p_val = stats.shapiro(df_stand["average_time_per_user"])
print("Test-statistis:{}, p-value:{}".format(shapiro_test, p_val))
shapiro_test, p_val = stats.shapiro(df_stand["download"])
print("Test-statistis:{}, p-value:{}".format(shapiro_test, p_val))
shapiro_test, p_val = stats.shapiro(df_stand["install_base"])
print("Test-statistis:{}, p-value:{}".format(shapiro_test, p_val))
shapiro_test, p_val = stats.shapiro(df_stand["open_rate"])
print("Test-statistis:{}, p-value:{}".format(shapiro_test, p_val))
shapiro_test, p_val = stats.shapiro(df_stand["revenue"])
print("Test-statistis:{}, p-value:{}".format(shapiro_test, p_val))
shapiro_test, p_val = stats.shapiro(df_stand["total_active_days"])
print("Test-statistis:{}, p-value:{}".format(shapiro_test, p_val))
shapiro_test, p_val = stats.shapiro(df_stand["total_time"])
print("Test-statistis:{}, p-value:{}".format(shapiro_test, p_val))
shapiro_test, p_val = stats.shapiro(df_stand["genre"])
print("Test-statistis:{}, p-value:{}".format(shapiro_test, p_val))

### 게임 중 90주 이상 데이터가 쌓인 게임 추세만 확인하기
- 90주 이상도 마찬가지로 상당히 게임별로 추세가 다름
- 이걸로 보다 확실히 게임별 혹은 그룹화에 이유가 생김

In [ ]:
df2 = pd.DataFrame(columns = df.columns)
cnt = 90
for num in range(0,1000):
    co = df[(df["product_id"] == num)]
    size = co.count()
    if size.start_date >= cnt:
        print(co.count(), num)
        df2 = pd.concat([df2,co], axis=0)

In [ ]:
name = df2[['product_id']]
name = name.drop_duplicates()
name.reset_index(inplace=True)
name = name.drop(columns='index')
name['product_id'][0]
number = df2.start_date.count()/cnt
number
name.count()

In [ ]:
plt.figure(figsize=(50, 100))
f, axes = plt.subplots(int(number/5 + 1), 5)
# 격자 크기 설정
f.set_size_inches((50, 50))
# 격자 여백 설정
plt.subplots_adjust(wspace = 0.2, hspace = 0.2)

x = 0;
y = 0;

for num in (range(0,name['product_id'].count())):
    axes[y][x].plot(df2[(df2["product_id"] == name['product_id'][num])].start_date, df2[(df2["product_id"] == name['product_id'][num])].revenue)
    x = x + 1
    if x == 5:
        y = y + 1
        x = 0

#plt.show()
plt.savefig(r'C:\Users\heygu\savefig_default.png')

### 게임 중 90주 이상 데이터가 쌓인 연도별 추세 
- 90주 이상도 마찬가지로 게임별로 연도별 추세가 비슷한 것이 있고 없는 것이 있다.
- 이걸로 보다 확실히 게임별 혹은 그룹화에 이유가 생김
- 그래프를 보니 전년도에 급 상승하고 내년에는 상승없이 하락세인게 모바일 확률이 높다. 년도간 경향이 비슷한 애들이 온라인일 확률이 높을 거라 예상한다.

In [ ]:
fig = plt.figure(figsize=(30,60))

for store in range(0,name['product_id'].count()):
    storeset = df2[(df2["product_id"] == name['product_id'][store])]
    storeset_2021 = storeset[storeset.end_Year==2021]
    storeset_2022 = storeset[storeset.end_Year==2022]
    
    ax = fig.add_subplot(int(number/5 + 1), 5, store + 1)
    
    plt.title(f"store_{store}")
    ax.plot(storeset_2021.end_WeekOfYear, storeset_2021.revenue, label="2021", alpha=0.3)
    ax.plot(storeset_2022.end_WeekOfYear, storeset_2022.revenue, label="2022", alpha=0.3)
    ax.legend()
    
plt.show()

### 게임 중 90주 이상 데이터에서의 변수간 상관관계
- 마찬가지로 데이터가 다소 완전하더라도 모든 게임별 변수간 상관관계가 일치하지는 않는다.
- 다소 download, install_base, open_rate, total_active_days, total_time 양의 상관관계를 자주 보여 주지만 그렇지 않은 게임도 다수 존재하는 편이다. 
- 그룹화에 필요성이 느껴진다. 

In [ ]:
corr = []
for num in range(0,name['product_id'].count()):
    co = df[df.product_id==name['product_id'][num]]
    co = co.reset_index()
    num_corr = co.corr(method='spearman', numeric_only=True)['revenue']
    num_corr = num_corr.drop(['index','revenue', 'start_Year', 'start_Month', 'start_Day', 'strart_WeekOfYear'])
    corr.append(num_corr)
corr_df = pd.concat(corr, axis=1).T
corr_df.index = list(range(0,name['product_id'].count()))
f, ax = plt.subplots(figsize=(30,15))
plt.title("게임별 매출액과 변수들간의 상관관계", fontsize=15)
sns.heatmap(corr_df.T, cmap=sns.diverging_palette(240,10,as_cmap=True), ax=ax, annot = True)
plt.xlabel('게임(id)')
plt.show()

### Box Plot
- 이상치 확인
- 상당히 이상치가 많게 느껴짐
- 이거는 게임별 특성을 고려하지 않은 상태에서 체크한 것이므로 그룹화후 다시 확인하겠음

In [ ]:
%matplotlib inline
col_list = list(['average_active_days', 'average_active_users', 
                     'average_session_duration', 'average_session_per_user', 'average_time_per_user', 'total_active_days', 'total_time', 'download', 'install_base', 'open_rate', 'genre','revenue'])
for column in col_list:
    fig = plt.figure(figsize=(20,5))
    sns.boxplot(x = df[column], color='yellow')
    plt.show()

## 게임별 특성에 따른 그룹화 진행

### 시계열 기준으로 1차 그룹화
- 최소 50개 이상의 값이 존재하는 경우 그룹화 수행(추세가 보여야 그룹화 가능)
- 앞에서 채워야함 . 보간법 채택이요 경향이나 추세가 사라지기 원하지 않음
- 온라인 모바일에 경우 시각적으로 보이는 경향이 존재했기에
- 시계열(Time Series) 데이터나 연속된 수치를 가지는 데이터의 경우에는 우리는 일종의 연속성있는 패턴을 발견할 수 있습니다. 이런 경우 보간(Interpolation)을 통해 앞,뒤 값을 통하여 유추하여 좀 더 스마트하게 결측치(NaN)를 채워줄 수 있습니다.
- 하지만 게임산업에 특성상 잦은 이벤트나 각종 사건에 굉장히 예민하고 튀는 값들이 굉장히 많습니다. 기존적으로 추세만 봐도 그걸 느낄 수 있습니다. 그래서 그냥 중앙값으로 하는 것이 극단치에 영향을 덜 받을 것입니다.

In [1]:
df2 = pd.DataFrame(columns = df.columns)
cnt = 80
for num in range(0,1000):
    co = df[(df["product_id"] == num)]
    #co = df[(df["product_id"] == num)&(df["end_Year"] == 2021)]
    size = co.count()
    if size.start_date >= cnt:
        print(co.count(), num)
        df2 = pd.concat([df2,co], axis=0)

NameError: name 'pd' is not defined

In [ ]:
name = df2[['product_id']]
name = name.drop_duplicates()
name.reset_index(inplace=True)
name = name.drop(columns='index')
name['product_id'][0]
number = df2.start_date.count()/cnt
number
name.count()

In [1]:
from tslearn.clustering import TimeSeriesKMeans

scaled_time_series_df = pd.DataFrame()

for num in range(0, name['product_id'].count()) :
    # test 셋은 2022년 10월 데이터이기 때문에 각 연도별 10월(포함) 이전의 시계열 데이터의 유사성을 판단하여 type을 나누었습니다.
    time_series = df[(df.product_id==name['product_id'][num])&(df.end_Year==2021)]['revenue'].values.reshape(-1, 1)
    scaled_time_series = pd.DataFrame(time_series)
    scaled_time_series_df[name['product_id'][num]] = scaled_time_series
    scaled_time_series_df.loc[scaled_time_series_df[col_name] != scaled_time_series_df[col_name], col_name] = scaled_time_series_df[col_name].median()

OSError: [WinError 126] 지정된 모듈을 찾을 수 없습니다. Error loading "C:\Users\heygu\anaconda3\Lib\site-packages\torch\lib\caffe2_nvrtc.dll" or one of its dependencies.

In [ ]:
transpose_scaled_time_series_df = scaled_time_series_df.transpose()
    
km = TimeSeriesKMeans(n_clusters=3, 
                      metric="dtw", 
                      max_iter=5,
                      random_state=2022)

prediction = km.fit_predict(transpose_scaled_time_series_df)

list_0 = []
list_1 = []
list_2 = []
list_3 = []
list_4 = []
list_5 = []

for i in range(len(prediction)) :
    if prediction[i] == 0 :
        list_0.append(name["product_id"][i])
    elif prediction[i] == 1 :
        list_1.append(name["product_id"][i])
    elif prediction[i] == 2:
        list_2.append(name["product_id"][i])
    elif prediction[i] == 3:
        list_3.append(name["product_id"][i])
    elif prediction[i] == 4:
        list_4.append(name["product_id"][i])
    else:
        list_5.append(name["product_id"][i])

print("Clustering 0 : ", list_0)
print("Clustering 1 : ", list_1)
print("Clustering 2 : ", list_2)
print("Clustering 3 : ", list_3)
print("Clustering 4 : ", list_4)
print("Clustering 5 : ", list_5)

#### 그룹화된 게임별 추세 비교 
- 연도별 비교
- 전체 추세 비교
- 80개에 데이터가 남아있는 게임을 클러스터링 
- 3개로 클러스터링 : 가장 분배가 적절한 결과를 보였기 때문에
 (너무 지나치게 세분화되어 1개-2개씩 남는 경우는 배제 하였다. 이유는 그런 경우 모델링 시 과적합 우려가 되기 때문이다.)
 - 기준을 모바일적 성향에 게임의 그래프가 어느정도 분리되는 시점으로 클러스터링 갯수를 잡음
 <br>
 - 다시한번 분배해서 살펴보니 2가지 성향이 눈에 띄었는제 하나는 급격히 상승했다가 낮아지는 모바일 게임 곡선
 - 년도별 비슷한 추세를 나타내는 온라인 게임 성향이 보였다.
 
 <br>확실히 앞에 년도가 중요성이 크다. 앞에 년도가 40개 이상 남은 애들로 클러스터링을 시도하자.굉장히 앞년도를 비슷하게 분류했다.

In [ ]:
new2 = pd.DataFrame()

for i in range(0, len(list_2)) :
    new = pd.DataFrame()
    new["revenue"] = scaled_time_series_df[list_2[i]]
    new["product_id"] = list_2[i]
    df_new = df[(df["product_id"] == 13)]["end_Year"]
    df_new = df_new.reset_index()
    new["end_Year"] = df_new["end_Year"]
    df_new = df[(df["product_id"] == 13)]["end_WeekOfYear"]
    df_new = df_new.reset_index()
    new["end_WeekOfYear"] = df_new["end_WeekOfYear"]
    new2 = pd.concat([new2,new], axis=0)
    
#new2

In [ ]:
fig = plt.figure(figsize=(20,30))

for num in range(0,len(list_2)):
    storeset = new2[new2['product_id'] == list_2[num]]
    storeset_2021 = storeset[storeset.end_Year==2021]
    storeset_2022 = storeset[storeset.end_Year==2022]
    
    ax = fig.add_subplot(20,11, num+ 1)
    
    plt.title(f"store_{num}")
    ax.plot(storeset_2021.end_WeekOfYear, storeset_2021.revenue, label="2021", alpha=0.3)
    ax.plot(storeset_2022.end_WeekOfYear, storeset_2022.revenue, label="2022", alpha=0.3)
    ax.legend()
    
plt.show()

#### 그래프 겹쳐서 확인
- 앞에 2021년도 값이 40개 이상일때 가장 잘 클러스터링 됨
- 시간은 흐름이 중요한데 끊겨서 많은 것보다는 연속적인 시간의 흐름 값이 중요한 듯하다.

In [ ]:
fig = plt.figure(figsize=(30,30))

fig, ax = plt.subplots()
fig2, ax2 = plt.subplots()


for num in range(0,len(list_0)):
    storeset = new2[new2['product_id'] == list_0[num]]
    storeset_2021 = storeset[storeset.end_Year==2021]
    storeset_2022 = storeset[storeset.end_Year==2022]
    
    #ax = fig.add_subplot(20,11, num + 1)
    
   # plt.title(f"store_{num}")
    ax.plot(storeset_2021.end_WeekOfYear, storeset_2021.revenue, label="2021", alpha=0.3)
    ax2.plot(storeset_2022.end_WeekOfYear, storeset_2022.revenue, label="2022", alpha=0.3)
    #ax.legend()
    
plt.show()

### 시계열 기준으로 1차 그룹화
- 이유: 그래프가 겹치는 부분이 유사성과 관련해서는 그룹화가 되는 듯 보이나. 전체적인 추세를 제대로 반영하여 클러스터링이 된는 것 같지 않다.
- 연도별로 클러스터링을 진행해서 결측치에 대해서도 어느정도 회피해보려한다.

- 2021년 기준 클러스터링 (1년 52주로해서 40주 이상인 게임이 441개로 절반 정도 존재) 
- 2022년 기준 클러스터링 : 40주 이상인 값이 129개 밖에 안됨 클러스터링 하기에는 너무 적다.

#### 2021그룹화
- 2 - 3개가 적당하다 그 이상은 1-2개로 남는 경우가 생겨 과적합 우려가 있다. 특성을 너무 자세히 나눔
- 2개를 나누었을때 모바일 적 성향에 친구와 온라인 성향에 게임이 나눠어진 경향이 있었다.
- 다소 비슷한게 클러스터링 되었다.
- 그만큼 2021년도의 전반적인 추세의 중요성이 입증되었다.

In [ ]:
df2 = pd.DataFrame(columns = df.columns)
cnt = 40
for num in range(0,1000):
    #co = df[(df["product_id"] == num)&(df["end_Year"] == 2021)]
    co = df[(df["product_id"] == num)&(df["end_Year"] == 2022)]
    size = co.count()
    if size.start_date >= cnt:
        df2 = pd.concat([df2,co], axis=0)

In [ ]:
name = df2[['product_id']]
name = name.drop_duplicates()
name.reset_index(inplace=True)
name = name.drop(columns='index')
number = df2.start_date.count()/cnt
number
name.count()
name

In [ ]:
from tslearn.clustering import TimeSeriesKMeans

scaled_time_series_df = pd.DataFrame()

for num in range(0, name['product_id'].count()) :
    col_name = "Game " +str(num)
    time_series = df[(df.product_id==name['product_id'][num])&(df["end_Year"] == 2021)]['revenue'].values.reshape(-1, 1)
    scaled_time_series = pd.DataFrame(time_series)
    scaled_time_series_df[name['product_id'][num]] = scaled_time_series
    scaled_time_series_df.loc[scaled_time_series_df[name['product_id'][num]] != scaled_time_series_df[name['product_id'][num]], name['product_id'][num]] = scaled_time_series_df[name['product_id'][num]].median()

In [ ]:
transpose_scaled_time_series_df = scaled_time_series_df.transpose()
    
km = TimeSeriesKMeans(n_clusters=2, 
                      metric="dtw", 
                      max_iter=10,
                      random_state=0
                     ,n_jobs=-1)

prediction = km.fit_predict(transpose_scaled_time_series_df)

list_0 = []
list_1 = []
list_2 = []
list_3 = []
list_4 = []
list_5 = []

for i in range(len(prediction)) :
    if prediction[i] == 0 :
        list_0.append(name["product_id"][i])
    elif prediction[i] == 1 :
        list_1.append(name["product_id"][i])
    elif prediction[i] == 2:
        list_2.append(name["product_id"][i])
    elif prediction[i] == 3:
        list_3.append(name["product_id"][i])
    elif prediction[i] == 4:
        list_4.append(name["product_id"][i])
    else:
        list_5.append(name["product_id"][i])

print("Clustering 0 : ", list_0)
print("Clustering 1 : ", list_1)
print("Clustering 2 : ", list_2)
print("Clustering 3 : ", list_3)
print("Clustering 4 : ", list_4)
print("Clustering 5 : ", list_5)

In [ ]:
new2 = pd.DataFrame()

for i in range(0, len(list_1)) :
    new = pd.DataFrame()
    new["revenue"] = scaled_time_series_df[list_1[i]]
    new["product_id"] = list_1[i]
    df_new = df[(df["product_id"] == 13)]["end_Year"]
    df_new = df_new.reset_index()
    new["end_Year"] = df_new["end_Year"]
    df_new = df[(df["product_id"] == 13)]["end_WeekOfYear"]
    df_new = df_new.reset_index()
    new["end_WeekOfYear"] = df_new["end_WeekOfYear"]
    new2 = pd.concat([new2,new], axis=0)
    

In [ ]:
fig = plt.figure(figsize=(30,30))

fig, ax = plt.subplots()

for num in range(0,len(list_0)):
    storeset = new2[new2['product_id'] == list_0[num]]
    storeset_2021 = storeset[storeset.end_Year==2021]
   # storeset_2022 = storeset[storeset.end_Year==2022]
    
    #ax = fig.add_subplot(20,11, num + 1)
    
    #plt.title(f"store_{num}")
    ax.plot(storeset_2021.end_WeekOfYear, storeset_2021.revenue, label="2021", alpha=0.3)
   # ax.plot(storeset_2022.end_WeekOfYear, storeset_2022.revenue, label="2022", alpha=0.3)
    #ax.legend()
    
plt.show()

In [ ]:
fig = plt.figure(figsize=(30,30))

fig, ax = plt.subplots()

for num in range(0,len(list_1)):
    storeset = new2[new2['product_id'] == list_1[num]]
    storeset_2021 = storeset[storeset.end_Year==2021]
   # storeset_2022 = storeset[storeset.end_Year==2022]
    
    #ax = fig.add_subplot(20,11, num + 1)
    
    #plt.title(f"store_{num}")
    ax.plot(storeset_2021.end_WeekOfYear, storeset_2021.revenue, label="2021", alpha=0.3)
   # ax.plot(storeset_2022.end_WeekOfYear, storeset_2022.revenue, label="2022", alpha=0.3)
    #ax.legend()
    
plt.show()

In [ ]:
fig = plt.figure(figsize=(30,30))

#fig, ax = plt.subplots()

for num in range(0,len(list_0)):
    storeset = new2[new2['product_id'] == list_0[num]]
    storeset_2021 = storeset[storeset.end_Year==2021]
    storeset_2022 = storeset[storeset.end_Year==2022]
    
    ax = fig.add_subplot(15,10, num + 1)
    
    #plt.title(f"store_{num}")
    ax.plot(storeset_2021.end_WeekOfYear, storeset_2021.revenue, label="2021", alpha=0.3)
    ax.plot(storeset_2022.end_WeekOfYear, storeset_2022.revenue, label="2022", alpha=0.3)
    ax.legend()
    
plt.show()

### 최종 결정
- 앞에 주기는 30이상이고 전체 주는 70이상인 값 / 60 , 30 /전체 40개
- 확실히 데이터가 많을 수록 좋음
- 클러스터링은 2 - 3개로 확정
- 가장 잘 그래프 상 추세를 분리함

## 그룹화 잘 분배 되었는지 검증
- 2가지 경우(전체 그룹화 / 2021년만 그룹화)
- 상관관계 분석하기
- 상자그림 
- 산점도 분석
- 시계열 분해해서 보기

### 그룹별 상자그림

In [ ]:
%matplotlib inline
col_list = list(['average_active_days', 'average_active_users', 

                 'average_session_duration', 'average_session_per_user', 'average_time_per_user', 'total_active_days', 'total_time', 'download', 'install_base', 'open_rate', 'genre','revenue'])

df_box = df[(df["Type"]==0)]

for column in col_list:
    fig = plt.figure(figsize=(20,5))
    sns.boxplot(x = df_box[column], color='yellow')
    plt.show()

### 그룹별 상관관계
- 그룹별 상관관계
- 그룹화한 게임별 상관관계
- 그룹화한 연도별 분석

In [ ]:
corr = []
for num in range(0,3):
    co = df[df.Type==num]
    co = co.reset_index()
    num_corr = co.corr(method='spearman')['revenue']
    num_corr = num_corr.drop(['index','revenue', 'start_Year', 'start_Month', 'start_Day', 'strart_WeekOfYear'])
    corr.append(num_corr)
corr_df = pd.concat(corr, axis=1).T
corr_df.index = list(range(0,3))
f, ax = plt.subplots(figsize=(30,15))
plt.title("게임별 매출액과 변수들간의 상관관계", fontsize=15)
sns.heatmap(corr_df.T, cmap=sns.diverging_palette(240,10,as_cmap=True), ax=ax, annot = True)
plt.xlabel('게임(id)')
plt.show()

In [ ]:
corr = []
lentgh = 0
for num in range(0,100):
    co = df[(df.Type==0)&(df.product_id==num)]
    if len(co) > 0:
        lentgh = lentgh + 1
        co = co.reset_index()
        num_corr = co.corr(method='spearman')['revenue']
        num_corr = num_corr.drop(['index','revenue', 'start_Year', 'start_Month', 'start_Day', 'strart_WeekOfYear'])
        corr.append(num_corr)
    
corr_df = pd.concat(corr, axis=1).T
corr_df.index = list(range(0,lentgh))
f, ax = plt.subplots(figsize=(30,15))
plt.title("게임별 매출액과 변수들간의 상관관계", fontsize=15)
sns.heatmap(corr_df.T, cmap=sns.diverging_palette(240,10,as_cmap=True), ax=ax, annot = True)
plt.xlabel('게임(id)')
plt.show()

lentgh

In [ ]:
corr = []
lentgh = 0
for num in range(0,100):
    co = df[(df.Type==0)&(df.end_Year==2021)]
    if len(co) > 0:
        lentgh = lentgh + 1
        co = co.reset_index()
        num_corr = co.corr(method='spearman')['revenue']
        num_corr = num_corr.drop(['index','revenue', 'start_Year', 'start_Month', 'start_Day', 'strart_WeekOfYear'])
        corr.append(num_corr)
    
corr_df = pd.concat(corr, axis=1).T
corr_df.index = list(range(0,lentgh))
f, ax = plt.subplots(figsize=(30,15))
plt.title("게임별 매출액과 변수들간의 상관관계", fontsize=15)
sns.heatmap(corr_df.T, cmap=sns.diverging_palette(240,10,as_cmap=True), ax=ax, annot = True)
plt.xlabel('게임(id)')
plt.show()

lentgh

### 그룹별 산점도

In [ ]:
sns.set(style='whitegrid') # 배경을 하얗게 한다
df_stand = df
sns.pairplot(df_stand[['average_active_days', 'average_active_users', 
                     'average_session_duration', 'average_session_per_user', 'average_time_per_user', 'total_active_days', 'total_time', 'download', 'install_base', 'open_rate', 'genre', 'revenue']])
plt.show()

In [ ]:
sns.set(style='whitegrid') # 배경을 하얗게 한다

df_stand = df[(df.Type==0)]

sns.pairplot(df_stand[['average_active_days', 'average_active_users', 
                     'average_session_duration', 'average_session_per_user', 'average_time_per_user', 'total_active_days', 'total_time', 'download', 'install_base', 'open_rate', 'genre', 'revenue']])
plt.show()

## 그외 데이터분석에 도움이 될 작업

### 변수간 상관관계 분석으로 revenue에 Type 주는 영향 보기
- 나름 영향을 주고 있다.

In [ ]:
plt.figure(figsize=(28,14))
plt.xticks( fontsize=20)
plt.yticks( fontsize=20)
temp = df[df.end_Year==2021]
sns.heatmap(df.corr(method='spearman'), cmap='Reds', annot=True, annot_kws={'size':12})
plt.title('Correlation Matrix only 2021', fontsize=30)

In [ ]:
plt.figure(figsize=(28,14))
plt.xticks(fontsize=20) 
plt.yticks(fontsize=20)
temp = df[df.end_Year==2022]
sns.heatmap(df.corr(method='spearman'), cmap='Reds', annot=True, annot_kws={'size':12})
plt.title('Correlation Matrix only 2022', fontsize=30)

### 변수 특성 시각화
- 게임별 매출액 비교
- 시간에 따른 매출액 추이(달, 주)
- 상관관계가 큰 변수들의 시각화

In [ ]:
# Group by year
df.groupby('end_Year')['revenue'].mean()
# Group by month
df.groupby('end_Month')['revenue'].mean()
# Group by week
df.groupby('end_WeekOfYear')['revenue'].mean()

# Visualization by month
monthly_sales = pd.pivot_table(df, values = "revenue", columns = "end_Year", index = "end_Month")
monthly_sales.plot()

fig = sns.barplot(x='end_Month', y='revenue', data=df)


# Visualization by week
weekly_sales = pd.pivot_table(df, values = "revenue", columns = "end_Year", index = "end_WeekOfYear")
weekly_sales.plot()
plt.figure(figsize=(70,6))
fig = sns.barplot(x='end_WeekOfYear', y='revenue', data=df)

# External variable check

plt.figure(figsize=(5,3))
fig = sns.barplot(x="Type", y='revenue', data=df)

fuel_price = pd.pivot_table(df, values = "revenue", index= "total_active_days")
fuel_price.plot()


unemployment = pd.pivot_table(df, values = "revenue", index= "total_time")
unemployment.plot()

### 변수 추가
- 유의성을 보고 부족해서 채운거임 (결과보고 부족해서)
- 하루 매출
- 유저 당 매출
- 1주 2주, 3주,4주 만들기

#### 시계열 acf,pacf 분석
- 1 ~ 4주 사이에 영향력이 강해 보임
- 1 ~4주 매출액을 만들겠음

In [ ]:
def func(df):
    df_sum = {}
    df_sum['revenue'] = df['revenue'].sum()
    df_sum['total_active_days'] = df['total_active_days'].mean()
    df_sum['total_time'] = df['total_time'].mean()
    df_sum['average_time_per_user'] = df['average_time_per_user'].mean()
    return pd.Series(df_sum, index=['revenue', 'total_active_days', 
                                    'total_time', 'average_time_per_user'])

df_agg = df.groupby(['end_Year', 'end_date']).apply(func).reset_index()

df_agg.head()

In [ ]:
timeSeries_decomp = df_agg.loc[:, ["end_date","revenue"]]
timeSeries_decomp.index = timeSeries_decomp.end_date
ts_decomp = timeSeries_decomp.drop("end_date",axis=1)

ts_decomp.head()

In [ ]:
decomp = seasonal_decompose(ts_decomp['revenue'], model='additive', period=4)

fig = plt.figure()  
fig = decomp.plot()  
fig.set_size_inches(20, 15)
plt.show()

In [ ]:
# ACF, PACF 그래프 시각화

fig = plt.figure(figsize=(20,8))
ax1 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_acf(ts_decomp, lags=45, ax=ax1)

ax2 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_pacf(ts_decomp, lags=45, ax=ax2)

#### 평균 매출 열 추가

In [ ]:
df["Day_revenue"] = df["revenue"]/7
df["user_revenue"] = df["revenue"]/df["average_active_users"]

#### 1~4주전 열 추가

In [ ]:
df['revenue_1w'] = df['revenue'].groupby(df["product_id"]).shift(1)
df['revenue_2w'] = df['revenue'].groupby(df["product_id"]).shift(2)
df['revenue_3w'] = df['revenue'].groupby(df["product_id"]).shift(3)
df['revenue_4w'] = df['revenue'].groupby(df["product_id"]).shift(4)

## 전처리
- 활성도 0인거 그거 확인해서 중앙값 넣어주기 오케?
- 마지막으로 train, test 셋 나누어주기!
- 변수, 데이터 정제

### 로그변화
- 왼쪽 긴꼬리 분포를 띄고 있다.
- 매출이 치우쳐져 있다.
- 로그처리하면 소수점 값들이 너무 많아서 그냥 정규분포를 따르도록 변환하려함
- 하지만 애초에 필요가 없는듯..
- 매출은 애초에 한쪽에

In [ ]:
from scipy import stats

sns.distplot(df['revenue'], fit=stats.norm)

In [ ]:
sns.distplot(np.log1p(df['revenue']), fit=stats.norm)

In [ ]:
corr = []
for num in range(50,100):
    co = df[df.product_id==num]
    co = co.reset_index()
    num_corr = co.corr(method='spearman', numeric_only=True)['revenue']
    num_corr = num_corr.drop(['product_id','revenue', 'start_Year', 'start_Month', 'start_Day', 'strart_WeekOfYear'])
    corr.append(num_corr)
corr_df = pd.concat(corr, axis=1).T
corr_df.index = list(range(0,50))

f, ax = plt.subplots(figsize=(30,15))
plt.title("게임별 매출액과 변수들간의 상관관계", fontsize=15)
sns.heatmap(corr_df.T, cmap=sns.diverging_palette(240,10,as_cmap=True), ax=ax, annot = True)
plt.xlabel('게임(id)')
plt.show()

### 표준화 필요성 여부 판단
- 표준화 전과 후가 전혀 산점도 그래프와 상관관계 분석 결과에도 영향을 주지 않았다.
- 이미 0 ~1 사이의 값으로 변환된 상태였다.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
import pandas as pd

df_stand = df[['average_active_days', 'average_active_users', 
                     'average_session_duration', 'average_session_per_user', 'average_time_per_user', 'total_active_days', 'total_time', 'download', 'install_base', 'open_rate', 'genre', 'revenue']]
# 전체 컬럼 표준화

# 데이터 표준화 적용
StandardScaler = StandardScaler()
df_stand2 = StandardScaler.fit_transform(df_stand)

# 컬럼명 결합
df_stand = pd.DataFrame(data=df_stand2, columns=df_stand.columns)

df_new = df.drop(['average_active_days', 'average_active_users', 
                     'average_session_duration', 'average_session_per_user', 'average_time_per_user', 'total_active_days', 'total_time', 'download', 'install_base', 'open_rate', 'genre', 'revenue'], axis=1)

df_stand = pd.concat([df_new, df_stand], axis=1)
df_stand.head()

# 모델링/학습
1) GridSearch
2) CatBoostRegressor
3) LinearRegression 다중회귀모델 생성
4) RandomForestRegressor
5) STL-ETS
6) XGBRegressor
7) LGBMRegressor

## GridSearch

Grid는 격자라는 뜻. 일정한 범위(격자)를 지정해주면, 범위 내의 모든 조합을 비교해서 내가 정한 Score를 기준으로 하이퍼파라미터값을 구해준다. 단, '모든' 조합을 비교하기 때문에 느리다. 대신에 정확하다. 범위를 넓게 잡으면? 당연히 오래 걸린다.


In [ ]:
!pip install xgboost

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import metrics, ensemble, linear_model
from sklearn.ensemble import HistGradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
# X_train
X_train = df[df.end_Month <= 10]
y_train = X_train.revenue

feature = ['total_time', 'average_time_per_user', 'average_session_duration','average_session_per_user', 'average_active_users', 'average_active_days']

In [ ]:
parameters = {
              'objective':['reg:squarederror'],
              'learning_rate':[0.1], #so called `eta` value
              'max_depth': [50],
              'min_child_weight': [4],
              'subsample': [0.8],
              'colsample_bytree': [0.8],
              'n_estimators':[30000]
              } 

xgb = XGBRegressor(random_state = 2022)


xgb_grid = GridSearchCV(estimator=xgb,
                        param_grid =parameters,
                        cv = 10,
                        n_jobs=1,
                        scoring = 'neg_mean_absolute_error',
                        verbose=3,
                        
                        )

xgb_grid.fit(X_train[feature], y_train)
best_model = xgb_grid.best_estimator_

print("BEST SCORE : {}".format(xgb_grid.best_score_))
print("BEST PARAMETER : {}".format(xgb_grid.best_params_))

## CatBoost
CatBoost는 그라디언트 부스팅 알고리즘을 기반으로 한 머신러닝 프레임워크입니다. CatBoost는 특히 시계열 데이터 처리에 탁월한 성능을 보여줍니다. 